# Testing a preparation pipeline for mixture simulations
Creating a pure python setup, whose inputs are the drug molecule and dye, numbers of each, volume, and the outputs are the parameterized molecules simulation starting structures.

## 1. Parameterizing with OpenEye

In [34]:
import sys
sys.path.append("/Users/rossg/Work/nanoparticles/pipeline")
#from openmoltools import openeye         # for Choderalab wrappers of openeye software
import openeye_mod as openeye             # personal modification of antechamber file generation function
from openeye.oechem import *             # the whole of the openeye toolbox

First, loading the molecule as an openeye molecule object. Here, the molecule happens to be a mol2, but more filetypes are supported http://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html#chapter-molreadwrite

In [70]:
mol = OEGraphMol()           # initialising molecule object
ifs = oemolistream()         # initialising input stream for reading in data
ifs.SetFormat(OEFormat_MOL2) # specifying mol2 format input
ifs.open('1E.mol2')           
if OEReadMolecule(ifs,mol):  # this function automatically returns True or False, to help spot for errors. 
    pass
else:
    print "Problem loading molecule!"

Now that the molecule has been loaded, we can do things like add hydrogens and rename to the standard IUPAC scheme using the Choderalab wrapper.

In [71]:
mol = openeye.normalize_molecule(mol)

And add partial charges with openeye's method flavour of AM1-BCC, or skip straight to generating antechamber files with `oemol_to_antechamber`, which automatically adds charges.

In [6]:
#
openeye.oemol_to_antechamber(mol,'1e_gaff.mol2','1e.frcmod')

For the solvation of the system with `modeller`, I'll try creating an forcefield xml object of the molecule. This would negate the AMBER Leap setup stage.

I need to generate charges for the ligand first.

In [72]:
mol = openeye.get_charges(mol)
traj,ffxml = openeye.oemols_to_ffxml([mol])

/Users/rossg/Work/nanoparticles/pipeline


Hopefully, ffxml will work as an openmm forcefield object.

# 2. Packmol
Creating a box of a mixture of drugs and dyes. Packmol requires PDB files, so using OEChem to produce the pdbs for me. The starting structure is the gaff mol2 file produced above.

In [66]:
mol = OEGraphMol()           # initialising molecule object
ifs = oemolistream()         # initialising INPUT stream for reading in data
ofs = oemolostream()         # initialising the OUTPUT stream for writing data
# Reading:
ifs.SetFormat(OEFormat_MOL2) # specifying mol2 format input
#ifs.open('ie_gaff.mol2')           
ifs.open("MOL-0-0.gaff.mol2")
if OEReadMolecule(ifs,mol):  \
    pass
else:
    print "Problem loading molecule!"

# Writing:
ofs.SetFormat(OEFormat_PDB)      # specifying that I want a pdb format 
ofs.open('1e_gaff.pdb')
OEWriteMolecule(ofs,mol)

0

I'll use packmol to create a box of drug. The output is an mdtraj object. Either water is added here, or the box is solvated with OpenMM's `modeller`.

In [67]:
from openmoltools import packmol

In [68]:
box = packmol.pack_box(['1e_gaff.pdb'],[50], tolerance=2.0, box_size=160)
box.save('box.pdb')


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpjxO5gA.pdb
add_amber_ter


structure 1e_gaff.pdb
  number 50 
  inside box 0. 0. 0. 160.000000 160.000000 160.000000
end structure


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpjxO5gA.pdb
add_amber_ter


structure 1e_gaff.pdb
  number 50 
  inside box 0. 0. 0. 160.000000 160.000000 160.000000
end structure



## 3. Solvation with OpenMM's `modeller`

In [25]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [73]:
pdb = PDBFile('box.pdb')
forcefield = ForceField(ffxml,'tip3p.xml')
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)

ValueError: No template found for residue 1 (MOL).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.

In [74]:
ffxml.read()

''

In [75]:
ffxml.readlines()

[]

In [76]:
s = traj[0]

In [77]:
s.n_atoms

52

In [78]:
res = [res for res in s.topology.residues][0]


In [61]:
res.name

'MOL'